In [1]:
import os
import time
import math
import requests
import bs4 as bs
import numpy as np
import pandas as pd
import pickle as pkl
import datetime as dt
import tensorflow as tf
from sklearn import metrics
from matplotlib import style
import yahoo_finance as yahoo
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.finance import candlestick_ohlc
from IPython.display import display, Math, Latex
style.use('ggplot')

C:\Users\Khanh\Anaconda3\envs\py35\lib\site-packages\matplotlib\cbook.py:136: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [2]:
df = pd.read_csv('./ibm/full_ibm.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.head()

,Open,High,Low,Close,Volume,Adj Close,NDX,DJIA,SP500
Date,,,,,,,,,
2000-03-01,112.4375,116.0000,111.875,116.0000,10347700,87.761136,3790.550049,11357.50977,1455.219971
2000-04-01,114.0000,114.5000,110.875,112.0625,8227800,84.782175,3546.199951,10997.92969,1399.420044
2000-05-01,112.9375,119.7500,112.125,116.0000,12733200,87.761136,3507.310059,11122.65039,1402.109985
2000-06-01,118.0000,118.9375,113.500,114.0000,7971900,86.248013,3340.810059,11253.25977,1403.449951
2000-07-01,117.2500,117.9375,110.625,113.5000,11856700,85.869732,3529.600098,11522.55957,1441.469971


In [3]:
df.tail()

,Open,High,Low,Close,Volume,Adj Close,NDX,DJIA,SP500
Date,,,,,,,,,
2016-12-23,167.000000,167.490005,166.449997,166.710007,1701200,165.402189,4940.020020,19933.81055,2263.790039
2016-12-27,166.979996,167.979996,166.850006,167.139999,1397500,165.828809,4965.810059,19945.03906,2268.879883
2016-12-28,167.289993,167.740005,166.000000,166.190002,1757500,164.886264,4926.290039,19833.67969,2249.919922
2016-12-29,166.020004,166.990005,166.000000,166.600006,1663500,165.293051,4918.279785,19819.77930,2249.260010
2016-12-30,166.440002,166.699997,165.500000,165.990005,2952800,164.687836,4863.620117,19762.59961,2238.830078


In [13]:
def calculate_technical_indicator(data):
    data['ewma_5'] = data['Close'].ewm(span=5).mean()
    data['ewma_22'] = data['Close'].ewm(span=22).mean()
    data['ewma_200'] = data['Close'].ewm(span=200).mean()
    data['MACD'] = data['Close'].ewm(span=12).mean() - data['Close'].ewm(span=26).mean()
    data['Lower band'] = data['Close'].rolling(window=20).mean() - 2*data['Close'].rolling(window=20).std()
    data['Upper band'] = data['Close'].rolling(window=20).mean() + 2*data['Close'].rolling(window=20).std()
    data['Middle band'] = data['Close'].rolling(window=20).mean()
    data['AO'] = ((data['High'] + data['Low'])/2).rolling(window=5).mean() - ((data['High'] + data['Low'])/2).rolling(window=34).mean()
    data['AC'] =  data['AO'] - data['AO'].rolling(window=5).mean()
    data['%R'] = (data['High'].max() - data['Close'])*100/(data['High'].max()-data['Low'].min())
    obv = np.array(data['Close'])
    obv[0] = 0 
    for i in range(1, obv.shape[0]):
        if data['Close'][i] > data['Close'][i-1]:
            obv[i] = obv[i-1] + data['Volume'][i]
        elif data['Close'][i] < data['Close'][i-1]:
            obv[i] = obv[i-1] - data['Volume'][i]
        else:
            obv[i] = obv[i-1]
    data['OBV'] = pd.Series(obv, index=data.index)
#     typical_price = np.array((data['Open'] + data['High'] + data['Close'])/3)
#     raw_money_flow = np.array((data['Open'] + data['High'] + data['Close'])*data['Volume']/3)
#     mfi = np.array(data['Close'])
#     period = 14
#     mfi[:period] = np.nan
#     for i in range(1,data.shape[0] - period):
#         pos = 0
#         neg = 0
#         for k in range(period):
#             if data['Close'][i+k] <= data['Close'][i+k-1]:
#                 pos += raw_money_flow[i+k]
#             else:
#                 neg += raw_money_flow[i+k]
#         mfr = pos/neg
#         mfi[i+period] = 100 - 100/(1 + mfr)
#     data['MFI'] = pd.Series(mfi, index=data.index)
        
                
        
    return data
   
df = calculate_technical_indicator(df)
df.head()


,Open,High,Low,Close,Volume,Adj Close,NDX,DJIA,SP500,ewma_5,...,ewma_200,MACD,Lower band,Upper band,Middle band,AO,AC,%R,OBV,MFI
Date,,,,,,,,,,,,,,,,,,,,,
2000-03-01,112.4375,116.0000,111.875,116.0000,10347700,87.761136,3790.550049,11357.50977,1455.219971,116.000000,...,116.000000,0.000000,NaN,NaN,NaN,NaN,NaN,61.708565,0.0,NaN
2000-04-01,114.0000,114.5000,110.875,112.0625,8227800,84.782175,3546.199951,10997.92969,1399.420044,113.637500,...,114.021406,-0.088341,NaN,NaN,NaN,NaN,NaN,64.140772,-8227800.0,NaN
2000-05-01,112.9375,119.7500,112.125,116.0000,12733200,87.761136,3507.310059,11122.65039,1402.109985,114.756579,...,114.687544,0.009537,NaN,NaN,NaN,NaN,NaN,61.708565,4505400.0,NaN
2000-06-01,118.0000,118.9375,113.500,114.0000,7971900,86.248013,3340.810059,11253.25977,1403.449951,114.442308,...,114.513071,-0.018389,NaN,NaN,NaN,NaN,NaN,62.943972,-3466500.0,NaN
2000-07-01,117.2500,117.9375,110.625,113.5000,11856700,85.869732,3529.600098,11522.55957,1441.469971,114.080569,...,114.306384,-0.053072,NaN,NaN,NaN,NaN,NaN,63.252824,-15323200.0,NaN


In [5]:
df.tail()

,Open,High,Low,Close,Volume,Adj Close,NDX,DJIA,SP500,ewma_5,ewma_22,ewma_200,MACD,Lower band,Upper band,Middle band,AO,AC,%R
Date,,,,,,,,,,,,,,,,,,,
2016-12-23,167.000000,167.490005,166.449997,166.710007,1701200,165.402189,4940.020020,19933.81055,2263.790039,167.007419,164.756987,155.579160,2.168195,158.984878,170.955123,164.970000,4.496765,-1.296082,30.384822
2016-12-27,166.979996,167.979996,166.850006,167.139999,1397500,165.828809,4965.810059,19945.03906,2268.879883,167.051612,164.964205,155.694194,2.072298,159.043100,171.158900,165.101000,4.287883,-0.942588,30.119214
2016-12-28,167.289993,167.740005,166.000000,166.190002,1757500,164.886264,4926.290039,19833.67969,2249.919922,166.764409,165.070796,155.798629,1.897766,159.204593,171.263408,165.234000,3.827913,-0.852229,30.706031
2016-12-29,166.020004,166.990005,166.000000,166.600006,1663500,165.293051,4918.279785,19819.77930,2249.260010,166.709608,165.203771,155.906106,1.772104,159.568089,171.337912,165.453001,3.419237,-0.794693,30.452770
2016-12-30,166.440002,166.699997,165.500000,165.990005,2952800,164.687836,4863.620117,19762.59961,2238.830078,166.469740,165.272139,156.006443,1.604795,160.506784,171.016217,165.761501,2.960442,-0.838005,30.829570


In [ ]:
from sklearn import linear_model